In [2]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import os
import glob


from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE, ProjectionCARE
from helpers import save_8bit_tiff_imagej_compatible

try:
    from pathlib import Path
    Path().expanduser()
except (ImportError, AttributeError):
        from pathlib2 import Path

try:
        import tempfile
        tempfile.TemporaryDirectory
except (ImportError, AttributeError):
       from backports import tempfile    
        

from skimage import exposure

import time

os.environ["CUDA_VISIBLE_DEVICES"]="1"

**Movie 1**

In [3]:
basedir='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Training_Data_Sets/Training_CARE_projection/SpinwideFRAP4_Training_CARE_40x_bin2/Low' 

basedirResults3D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Training_Data_Sets/Training_CARE_projection/SpinwideFRAP4_Training_CARE_40x_bin2/Restored'
basedirResults2D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/m_balakireva/Maria_Movie2Lucas/Pcna4a/Projected'
basedirResults3Dextended= basedirResults3D + '/Restored'
basedirResults2Dextended= basedirResults2D + '/Projected'

Model_Dir='/run/media/sancere/DATA1/Lucas_NextonCreated_npz/'

In [4]:
RestorationModel ='Training_CARE_restoration_SpinwideFRAP4_Bin2'
ProjectionModel ='CARE_projection_SpinWideFRAP4_Bin1_3Gfp'

RestorationModel = CARE(config = None, name = RestorationModel, basedir = Model_Dir)
ProjectionModel = ProjectionCARE(config = None, name = ProjectionModel, basedir = Model_Dir) 


Loading network weights from 'weights_best.h5'.


In [5]:
Path(basedirResults3D).mkdir(exist_ok = True)
Path(basedirResults2D).mkdir(exist_ok = True)

Raw_path = os.path.join(basedir, '*TIF') #tif or TIF be careful

axes = 'ZYX'  #projection axes : 'YX'

filesRaw = glob.glob(Raw_path)

In [6]:
for fname in filesRaw:
       if  os.path.exists(fname) == True :
            if  os.path.exists(basedirResults3Dextended + os.path.basename(fname)) == False : #os.path.exists(basedirResults2Dextended + '_' + os.path.basename(fname)) == False  or
                print(fname)
                y = imread(fname)
                restored = RestorationModel.predict(y, axes, n_tiles = (1,2,4)) #n_tiles is for the decomposition of the image in (z,y,x). (1,2,2) will work with light images. Less tiles we have, faster the calculation is 
                projection = ProjectionModel.predict(restored, axes, n_tiles = (1,1,1)) #n_tiles is for the decomposition of the image in (z,y,x). There is overlapping in the decomposition wich is managed by the program itself
                axes_restored = axes.replace(ProjectionModel.proj_params.axis, '')
                restored = restored.astype('uint8') # if prediction and projection running at the same time
                #restored = restored.astype('uint16') # if projection training set creation or waiting for a future projection 
                projection = projection.astype('uint8')
                #save_tiff_imagej_compatible((basedirResults3Dextended  + os.path.basename(fname)) , restored, axes)
                save_tiff_imagej_compatible((basedirResults2Dextended + '_' + os.path.basename(fname)) , projection, axes_restored)